In [1]:
from Go2Py.sim.mujoco import Go2Sim
import numpy as np

In [2]:
robot = Go2Sim()
robot.standUpReset()

In [3]:
from Go2Py.controllers.walk_these_ways import *

In [4]:
checkpoint_path = "/home/rstaion/projects/rooholla/locomotion/Go2Py/Go2Py/assets/checkpoints/walk_these_ways/"

cfg = loadParameters(checkpoint_path)
policy = Policy(checkpoint_path)
command_profile = CommandInterface()

In [5]:
agent = WalkTheseWaysAgent(cfg, command_profile, robot)
agent = HistoryWrapper(agent)

p_gains: [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]


In [6]:
control_dt = cfg["control"]["decimation"] * cfg["sim"]["dt"]
simulation_dt = robot.dt
obs = agent.reset()

In [7]:
robot.reset()
obs = agent.reset()
for i in range(5000):
    policy_info = {}
    action = policy(obs, policy_info)
    if i % (control_dt // simulation_dt) == 0:
        obs, ret, done, info = agent.step(action)
    robot.step()
    command_profile.yaw_vel_cmd = 1.2
    command_profile.x_vel_cmd = 0.8
    command_profile.y_vel_cmd = 0.0
    command_profile.stance_width_cmd=0.2
    command_profile.footswing_height_cmd=-0.05
    command_profile.step_frequency_cmd = 2.5
    time.sleep(robot.dt/4)

frq: 12.822735624382831 Hz
frq: 47.954632763194 Hz
frq: 53.309743511528 Hz
frq: 54.65674559220214 Hz
frq: 45.29730547005778 Hz
